In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
!unzip "/content/gdrive/MyDrive/4-condition-splited-data.zip"

Streaming output truncated to the last 5000 lines.
  inflating: 4-condition-splited-data/train/lung_benign tissue/lungn3836.jpeg  
  inflating: 4-condition-splited-data/train/lung_benign tissue/lungn3839.jpeg  
  inflating: 4-condition-splited-data/train/lung_benign tissue/lungn384.jpeg  
  inflating: 4-condition-splited-data/train/lung_benign tissue/lungn3840.jpeg  
  inflating: 4-condition-splited-data/train/lung_benign tissue/lungn3841.jpeg  
  inflating: 4-condition-splited-data/train/lung_benign tissue/lungn3843.jpeg  
  inflating: 4-condition-splited-data/train/lung_benign tissue/lungn3847.jpeg  
  inflating: 4-condition-splited-data/train/lung_benign tissue/lungn3849.jpeg  
  inflating: 4-condition-splited-data/train/lung_benign tissue/lungn3851.jpeg  
  inflating: 4-condition-splited-data/train/lung_benign tissue/lungn3852.jpeg  
  inflating: 4-condition-splited-data/train/lung_benign tissue/lungn3854.jpeg  
  inflating: 4-condition-splited-data/train/lung_benign tissue/lungn38

In [3]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


dataset_root = '4-condition-splited-data'


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(root=dataset_root + '/train', transform=transform)
test_dataset = ImageFolder(root=dataset_root + '/test', transform=transform)
val_dataset = ImageFolder(root=dataset_root+"/validation",transform=transform)


batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [4]:
import torch
import torch.nn as nn

class DeepCNNWithBatchNormAndDropout(nn.Module):
    def __init__(self,dropoutValue):
        super(DeepCNNWithBatchNormAndDropout, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3)
        self.batch_norm2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3)
        self.batch_norm3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3)
        self.batch_norm4 = nn.BatchNorm2d(512)
        #self.conv5 = nn.Conv2d(256, 512, kernel_size=3)
        #self.batch_norm5 = nn.BatchNorm2d(512)
        #self.conv6 = nn.Conv2d(512, 512, kernel_size=3)
        #self.batch_norm6 = nn.BatchNorm2d(512)

        # Max pooling layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        #RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x51200 and 131072x4096)
        #RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x61952 and 51200x4096)
        #RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x73728 and 61952x4096)
        # Fully connected layers
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512 * 12 * 12, 2048)
        self.batch_norm_fc1 = nn.BatchNorm1d(2048)
        self.dropout1 = nn.Dropout(dropoutValue)
        self.fc2 = nn.Linear(2048, 2048)
        self.batch_norm_fc2 = nn.BatchNorm1d(2048)
        self.dropout2 = nn.Dropout(dropoutValue)
        self.fc3 = nn.Linear(2048, len(train_dataset.classes))
        # Activation function
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.batch_norm2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.batch_norm3(self.conv3(x)))
        x = self.relu(self.batch_norm4(self.conv4(x)))
        x = self.pool(x)
        #x = self.relu(self.batch_norm5(self.conv5(x)))
        #x = self.relu(self.batch_norm6(self.conv6(x)))
        x = self.pool(x)
        # Flatten the output for the fully connected layers
        x = self.flatten(x)
        x = self.dropout1(self.relu(self.batch_norm_fc1(self.fc1(x))))
        x = self.dropout2(self.relu(self.batch_norm_fc2(self.fc2(x))))
        x = self.fc3(x)
        return x


In [5]:
parameters = { "epochs" : [10,15],
               "dropout" : [0.4,0.6],
               "learning_rate" : [0.001,0.0001]}

In [6]:
def generate_combinations(params, current_combination, all_combinations):
    if not params:
        all_combinations.append(dict(current_combination))
        return
    current_param = list(params.keys())[0]
    for value in params[current_param]:
        current_combination[current_param] = value
        generate_combinations({k: v for k, v in params.items() if k != current_param}, current_combination, all_combinations)

all_combinations = []
generate_combinations(parameters, {}, all_combinations)

all_combinations

[{'epochs': 10, 'dropout': 0.4, 'learning_rate': 0.001},
 {'epochs': 10, 'dropout': 0.4, 'learning_rate': 0.0001},
 {'epochs': 10, 'dropout': 0.6, 'learning_rate': 0.001},
 {'epochs': 10, 'dropout': 0.6, 'learning_rate': 0.0001},
 {'epochs': 15, 'dropout': 0.4, 'learning_rate': 0.001},
 {'epochs': 15, 'dropout': 0.4, 'learning_rate': 0.0001},
 {'epochs': 15, 'dropout': 0.6, 'learning_rate': 0.001},
 {'epochs': 15, 'dropout': 0.6, 'learning_rate': 0.0001}]

In [7]:
import torch
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'
INPUT_SIZE = (224,224,3)

W,H,C = INPUT_SIZE
input_size = W*H*C
best_model_acc = 0
for id,param in enumerate(all_combinations):
    first_time = time.time()
    model = DeepCNNWithBatchNormAndDropout(param["dropout"]).to(device)

    losS = nn.CrossEntropyLoss()
    optim = torch.optim.Adam(model.parameters(), lr=param["learning_rate"])
    print(f'Epoch : {param["epochs"]} - Dropout : {param["dropout"]} - Learning Rate : {param["learning_rate"]}')
    for epoch in range(param["epochs"]):
        model.train()
        for inputs,labels in train_loader:
            inputs , labels = inputs.to(device),labels.to(device)
            optim.zero_grad()
            outputs = model(inputs)
            loss = losS(outputs,labels)
            loss.backward()
            optim.step()

        print(f'Epoch [{epoch + 1}/{param["epochs"]}], Training Loss: {loss.item()}')

        model.eval()
        test_loss = 0.0
        correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                test_loss += losS(outputs, labels).item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()

        test_loss /= len(val_loader.dataset)
        accuracy = correct / len(val_loader.dataset) * 100.0

        print(f'Test Loss: {test_loss}, Test Accuracy: {accuracy}%')

        if accuracy> best_model_acc:
            best_model_acc = accuracy
            torch.save(model.state_dict(), "cnn-condition-4-best-model.pth")
            print("Best model saved")
    last_time = time.time()
    print(f"\nModel {id+1} fit completed {last_time-first_time:.4f}\n")



Epoch : 10 - Dropout : 0.4 - Learning Rate : 0.001
Epoch [1/10], Training Loss: 0.1263812780380249
Test Loss: 0.007134374236383547, Test Accuracy: 83.07692307692308%
Best model saved
Epoch [2/10], Training Loss: 0.14707691967487335
Test Loss: 0.003243550759164067, Test Accuracy: 92.1025641025641%
Best model saved
Epoch [3/10], Training Loss: 0.06376083195209503
Test Loss: 0.003341800671724531, Test Accuracy: 92.2051282051282%
Best model saved
Epoch [4/10], Training Loss: 0.44090983271598816
Test Loss: 0.00525681136510311, Test Accuracy: 88.51282051282051%
Epoch [5/10], Training Loss: 0.007442773319780827
Test Loss: 0.005840636200796908, Test Accuracy: 89.84615384615384%
Epoch [6/10], Training Loss: 0.10166262835264206
Test Loss: 0.014122359791804345, Test Accuracy: 77.02564102564102%
Epoch [7/10], Training Loss: 0.03102480247616768
Test Loss: 0.0014539314892578821, Test Accuracy: 96.51282051282051%
Best model saved
Epoch [8/10], Training Loss: 0.011884326115250587
Test Loss: 0.00208520